# GFPGAN Colab Demo (Fork)

This Colab notebook installs dependencies, clones this fork, downloads pretrained weights automatically, and runs the GFPGAN inference script on sample images.

In [ ]:
%cd /content
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from IPython import get_ipython
import os
import sys
import subprocess
import tempfile
import shutil
import glob
import zipfile
from io import BytesIO
import base64
from google.colab import files
from IPython.display import Image as IPyImage

# Check if already installed
if not os.path.exists('/content/GFPGAN'):
    print("Installing GFPGAN...")

    # Clone and setup
    !git clone https://github.com/IAmJonoBo/GFPGAN.git
    %cd GFPGAN
    %pip install basicsr facexlib realesrgan
    %pip install -r requirements.txt
    !python setup.py develop

    # Create required directories
    !mkdir -p inputs/whole_imgs inputs/cropped_faces results

    print("Installation complete!")
else:
    print("GFPGAN already installed.")
    %cd GFPGAN

print("\n🎉 Setup complete! You can now upload images and run restoration.")

# UI Components
style = {'description_width': '120px'}
layout = widgets.Layout(width='500px')

# Upload widget
upload_widget = widgets.FileUpload(
    accept='image/*',
    multiple=True,
    description='Upload Images:',
    style=style,
    layout=layout
)

# URL input widget
url_widget = widgets.Text(
    value='',
    placeholder='https://example.com/image.jpg',
    description='Or Image URL:',
    style=style,
    layout=layout
)

# Download URL button
download_btn = widgets.Button(
    description='Download from URL',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

# Model version dropdown
version_widget = widgets.Dropdown(
    options=[('1.4 (Best)', '1.4'), ('1.3 (Natural)', '1.3'), ('1.2 (Sharp)', '1.2')],
    value='1.4',
    description='Model Version:',
    style=style,
    layout=layout
)

# Upscale factor
upscale_widget = widgets.IntSlider(
    value=2,
    min=1,
    max=4,
    step=1,
    description='Upscale Factor:',
    style=style,
    layout=layout
)

# Background upsampler
bg_widget = widgets.Dropdown(
    options=[('Real-ESRGAN', 'realesrgan'), ('None', 'none')],
    value='realesrgan',
    description='Background Enhance:',
    style=style,
    layout=layout
)

# Process button
process_btn = widgets.Button(
    description='🚀 Process Images',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

# Download results button
download_results_btn = widgets.Button(
    description='📦 Download Results',
    button_style='warning',
    layout=widgets.Layout(width='200px', height='40px')
)

uploads_dir = "/content/GFPGAN/inputs/whole_imgs"
results_dir = "/content/GFPGAN/results"

def handle_upload(change):
    """Handle file uploads"""
    clear_output(wait=True)

    for name, file_info in upload_widget.value.items():
        file_path = os.path.join(uploads_dir, name)
        with open(file_path, 'wb') as f:
            f.write(file_info['content'])
        print(f"Uploaded: {name}")

    show_ui()
    show_uploaded_files()

def download_from_url():
    """Download image from URL"""
    url = url_widget.value.strip()
    if not url:
        print("Please enter a URL")
        return

    try:
        import urllib.request
        filename = url.split('/')[-1] or 'downloaded_image.jpg'
        file_path = os.path.join(uploads_dir, filename)
        urllib.request.urlretrieve(url, file_path)
        print(f"Downloaded: {filename}")
        show_uploaded_files()
    except Exception as e:
        print(f"Error downloading: {e}")

def show_uploaded_files():
    """Display uploaded files"""
    print("\n" + "="*50)
    print("📁 INPUT FILES")
    print("="*50)

    # List uploaded/downloaded files
    files = glob.glob(os.path.join(uploads_dir, '*'))
    if files:
        print(f"\nFound {len(files)} input files:")
        for f in files[:3]:  # Show first 3
            name = os.path.basename(f)
            print(f"  {name}")
            try:
                display(IPyImage(filename=f, width=200))
            except:
                print("    (preview not available)")
        if len(files) > 3:
            print(f"  ... and {len(files) - 3} more")
    else:
        print("No input files found")

def process_images():
    """Run GFPGAN processing"""
    clear_output(wait=True)
    show_ui()

    # Check if there are input files
    input_files = glob.glob(os.path.join(uploads_dir, '*'))
    if not input_files:
        print("❌ No input files found. Please upload images first.")
        return

    print("🔄 Processing images...")
    print("This may take a few minutes depending on image size and quantity.\n")

    # Build command
    version = version_widget.value
    upscale = upscale_widget.value
    bg_upsampler = bg_widget.value

    cmd = [
        'python', 'inference_gfpgan.py',
        '-i', uploads_dir,
        '-o', results_dir,
        '-v', version,
        '-s', str(upscale),
        '--bg_upsampler', bg_upsampler
    ]

    try:
        # Run GFPGAN
        result = subprocess.run(cmd, capture_output=True, text=True, cwd='/content/GFPGAN')

        if result.returncode == 0:
            print("✅ Processing completed successfully!")
            print("\nCommand output:")
            print(result.stdout)
            show_results()
        else:
            print("❌ Processing failed!")
            print("Error:", result.stderr)
            print("Output:", result.stdout)

    except Exception as e:
        print(f"❌ Error running GFPGAN: {e}")

def show_results():
    """Display result files"""
    print("\n" + "="*50)
    print("🎨 RESULTS")
    print("="*50)

    # Find result files
    result_files = []
    for ext in ['*.jpg', '*.png', '*.jpeg']:
        result_files.extend(glob.glob(os.path.join(results_dir, '**', ext), recursive=True))

    if result_files:
        print(f"\nGenerated {len(result_files)} result files:")
        for f in result_files[:3]:  # Show first 3
            name = os.path.basename(f)
            print(f"  {name}")
            try:
                display(IPyImage(filename=f, width=300))
            except:
                print("    (preview not available)")
        if len(result_files) > 3:
            print(f"  ... and {len(result_files) - 3} more")
    else:
        print("No result files found")

def create_results_zip():
    """Create ZIP file of results for download"""
    result_files = []
    for ext in ['*.jpg', '*.png', '*.jpeg']:
        result_files.extend(glob.glob(os.path.join(results_dir, '**', ext), recursive=True))

    if not result_files:
        print("No result files to download")
        return

    # Create ZIP in memory
    zip_buffer = BytesIO()
    with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        for file_path in result_files:
            # Get relative path for ZIP structure
            rel_path = os.path.relpath(file_path, results_dir)
            zip_file.write(file_path, rel_path)

    zip_buffer.seek(0)

    # Download the ZIP file
    files.download('/content/gfpgan_results.zip')
    with open('/content/gfpgan_results.zip', 'wb') as f:
        f.write(zip_buffer.getvalue())

    print(f"📦 Created ZIP with {len(result_files)} files")

def show_ui():
    """Display the main UI"""
    display(HTML("<h2>🎭 GFPGAN Face Restoration</h2>"))

    # Upload section
    display(HTML("<h3>📤 Upload Images</h3>"))
    display(upload_widget)
    display(widgets.HBox([url_widget, download_btn]))

    # Settings section
    display(HTML("<h3>⚙️ Settings</h3>"))
    display(version_widget)
    display(upscale_widget)
    display(bg_widget)

    # Action buttons
    display(HTML("<h3>🚀 Actions</h3>"))
    display(widgets.HBox([process_btn, download_results_btn]))

# Wire up event handlers
upload_widget.observe(handle_upload, names='value')
download_btn.on_click(lambda x: download_from_url())
process_btn.on_click(lambda x: process_images())
download_results_btn.on_click(lambda x: create_results_zip())

# Show initial UI
show_ui()

In [ ]:
#@title Install dependencies
import os, sys, subprocess, shlex
if os.environ.get('NB_CI_SMOKE') == '1':
    print('NB_CI_SMOKE=1: skipping dependency install')
else:
    cmds = [
        [sys.executable, '-m', 'pip', 'install', '--upgrade', '--quiet', 'pip', 'setuptools', 'wheel'],
        ['pip', 'install', '--quiet', 'torch', 'torchvision', 'torchaudio'],
        ['pip', 'install', '--quiet', '--no-cache-dir', '--upgrade', 'git+https://github.com/xinntao/BasicSR@master', 'facexlib', 'realesrgan', 'opencv-python', 'tqdm', 'numpy', 'PyYAML', 'gfpgan'],
        ['pip', 'show', 'torch', 'torchvision', 'basicsr'],
    ]
    for c in cmds:
        print('$', ' '.join(shlex.quote(x) for x in c))
        subprocess.run(c, check=False)


In [ ]:
#@title Clone this fork
import os, subprocess
if os.environ.get('NB_CI_SMOKE') == '1':
    print('NB_CI_SMOKE=1: skipping clone')
else:
    if not os.path.isdir('GFPGAN'):
        subprocess.run(['git', 'clone', '--depth', '1', 'https://github.com/IAmJonoBo/GFPGAN.git'], check=False)
    os.chdir('GFPGAN')
    print('Repo at:', os.getcwd())
    subprocess.run(['ls', '-la', 'inputs'], check=False)


In [ ]:
#@title Run inference on sample images
import os, subprocess, sys
if os.environ.get('NB_CI_SMOKE') == '1':
    print('NB_CI_SMOKE=1: skipping inference')
else:
    os.chdir('GFPGAN')
    subprocess.run([sys.executable, 'inference_gfpgan.py', '-i', 'inputs/whole_imgs', '-o', 'results', '-v', '1.4', '-s', '2'], check=False)
    print('Results written to ./GFPGAN/results')


In [ ]:
# @title Display a few restored images
import glob
import os

from IPython.display import Image, display

base = "GFPGAN/results/restored_imgs"
imgs = sorted(glob.glob(os.path.join(base, "*")))[:4]
print(f"Displaying {len(imgs)} images from", base)
for p in imgs:
    display(Image(filename=p))

In [ ]:
# @title Upload your own images and run GFPGAN
# @markdown Upload one or more images; they will be processed with the settings below.
import os
import shutil
import subprocess
import sys

if os.environ.get('NB_CI_SMOKE') == '1':
    print('NB_CI_SMOKE=1: skipping upload UI')
else:
    from google.colab import files
    upload_dir = "GFPGAN/uploads"
    os.makedirs(upload_dir, exist_ok=True)
    uploaded = files.upload()
    for name, data in uploaded.items():
        # Save uploaded file and move into repo uploads folder
        with open(name, "wb") as f:
            f.write(data)
        shutil.move(name, os.path.join(upload_dir, os.path.basename(name)))
    # Inference parameters
    version = "1.4"  # @param ['1', '1.2', '1.3', '1.4']
    scale = 2  # @param {type: 'integer'}
    only_center_face = False  # @param {type: 'boolean'}
    aligned = False  # @param {type: 'boolean'}
    print("Running inference on uploads...")
    cmd = [sys.executable, "inference_gfpgan.py", "-i", "uploads", "-o", "results", "-v", str(version), "-s", str(scale)]
    if only_center_face:
        cmd.append("--only_center_face")
    if aligned:
        cmd.append("--aligned")
    subprocess.run(cmd, cwd="GFPGAN", check=False)
    print("Done. See GFPGAN/results")

Notes:
- GPU acceleration is optional; uncomment the CUDA wheel index in the install cell to use GPU on Colab.
- The inference script will automatically download the GFPGAN v1.4 weights if not present.
- For large batches, consider enabling the Real-ESRGAN background upsampler by keeping default settings (it is auto-disabled on CPU).